In [1]:
import os, pickle
import numpy as np
import pandas as pd
from keras.layers import Input, Conv2D, MaxPool2D, Reshape, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Set Directories 

pardir = os.path.dirname(os.getcwd())
PATH = os.path.join(pardir,"data\\caption_all_fa\\")
os.chdir(PATH)
SAVED = os.path.join(pardir,"data\\saved\\")


In [3]:
# Load data functions

def load_doc(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        text = file.read()
    return text

def load_clean_class(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1]
        if image_desc in dataset:
            if image_desc not in descriptions:
                descriptions[image_desc] = list()
            descriptions[image_desc].append(image_id)
    return descriptions

def load_class_dummy(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1]
        if image_desc in dataset:
            if image_desc not in descriptions:
                descriptions[image_desc] = list()
            image_id = dummies_dict[image_id]
            descriptions[image_desc].append(image_id)
    return descriptions

# Class to one_hot_vector dictionary
folder_names = sorted([entry_name for entry_name in os.listdir(PATH) if os.path.isdir(os.path.join(PATH, entry_name))])
dummies = pd.get_dummies(folder_names)
dummies_list = dummies.values.tolist()
dummies_dict = dict(zip(folder_names, dummies_list))


In [9]:
# Load train data
filename = SAVED + '/train_image.txt'
train = [line.split(',')[0][:-4] for line in open(filename, encoding='utf-8').read().splitlines()]

with open(SAVED + "train_image_features.pkl", "rb") as f: #choose image features _ENetB2
    train_features = pickle.load(f)

train_class = load_clean_class(SAVED + 'flower_class.txt', train)
class_dummy = load_class_dummy(SAVED + 'flower_class.txt', train)

# Load val data
filename = SAVED + '/val_image.txt'
val = [line.split(',')[0][:-4] for line in open(filename, encoding='utf-8').read().splitlines()]

with open(SAVED + "eval_image_features.pkl", "rb") as f: #choose image features _ENetB2
    val_features = pickle.load(f)

val_class = load_clean_class(SAVED + 'flower_class.txt', val)
val_class_dummy = load_class_dummy(SAVED + 'flower_class.txt', val)

# Load test data
filename = SAVED + '/test_image.txt'
test = [line.split(',')[0][:-4] for line in open(filename, encoding='utf-8').read().splitlines()]

with open(SAVED + "test_image_features.pkl", "rb") as f: #choose image features _ENetB2
    test_features = pickle.load(f)

test_class = load_clean_class(SAVED + 'flower_class.txt', test)
test_class_dummy = load_class_dummy(SAVED + 'flower_class.txt', test)

# Prepare data for training
def prepare_data(features, class_dummy):
    X_image, y_class = [], []
    for key in features.keys():
        X_image.append(features[key][0])
        key = key.replace(".jpg", "")
        if key in class_dummy:
            y_class.append(np.array(class_dummy[key][0]))
    return np.array(X_image), np.array(y_class)


In [10]:
X_image, y_class = prepare_data(train_features, class_dummy)
X_image_val, y_class_val = prepare_data(val_features, val_class_dummy)
X_image_test, y_class_test = prepare_data(test_features, test_class_dummy)

In [11]:
# Model definition
def Modified_m_CNN(DROP_OUT, DROP_OUT2, LAMBDA):
    inputs = Input(shape=(4096,))
    x = Reshape((16, 1, 256))(inputs)
    x = BatchNormalization()(x)
    conv_x = Conv2D(256, kernel_size=(14, 1), padding='valid', kernel_initializer='he_normal', activation='relu')(x)
    conv_x = Dropout(DROP_OUT2)(conv_x)
    max_x = MaxPool2D(pool_size=(2, 1))(conv_x)
    x = Flatten()(max_x)
    x = Dropout(DROP_OUT)(x)
    output = Dense(units=102, activation='softmax', kernel_regularizer='l2')(x)
    model = Model(inputs=inputs, outputs=output)
    return model


In [12]:
BATCH_SIZE = 128
EPOCHS = 40
LAMBDA = 0.05
DROP_OUT = 0.2
DROP_OUT2 = 0.4

accuracies = []


for i in range(10):
    print(f"Run {i+1}")

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model = Modified_m_CNN(DROP_OUT, DROP_OUT2, LAMBDA)

    # Model detailed settings
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Model fit
    history = model.fit(X_image, y_class, epochs=EPOCHS, batch_size=BATCH_SIZE, 
                        validation_data=(X_image_val, y_class_val), shuffle=True, 
                        callbacks=[early_stopping])

    # Model evaluation
    score = model.evaluate(X_image_test, y_class_test, verbose=1)
    accuracy = score[1] * 100
    accuracies.append(accuracy)
    print(f"Accuracy: {accuracy:.2f}%\n")

# Calculate mean and variance
mean_accuracy = np.mean(accuracies)
variance_accuracy = np.var(accuracies)

print(f"Accuracies: {', '.join(f'{acc:.2f}%' for acc in accuracies)}")
print(f"Mean Accuracy: {mean_accuracy:.2f}%")
print(f"Variance: {variance_accuracy:.2f}%")


Run 1
Epoch 1/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.1806 - loss: 5.7094 - val_accuracy: 0.5885 - val_loss: 2.8124
Epoch 2/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.5761 - loss: 2.8092 - val_accuracy: 0.7420 - val_loss: 2.0127
Epoch 3/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.7143 - loss: 1.9924 - val_accuracy: 0.7969 - val_loss: 1.6170
Epoch 4/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.7697 - loss: 1.5612 - val_accuracy: 0.8122 - val_loss: 1.4000
Epoch 5/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.8160 - loss: 1.2845 - val_accuracy: 0.8298 - val_loss: 1.2502
Epoch 6/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.8465 - loss: 1.1184 - val_accuracy: 0.8336 - val_loss: 1.1637
Epoch 7/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - accuracy: 0.8852 - loss: 0.9180 - val_accuracy: 0.8405 - val_loss: 1.0633
Epoch 8/40
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - accuracy: 0.8899 - loss: 0.8617 - val_accuracy: 0.

In [ ]:

#ENetB2
Accuracies: 93.45%, 94.30%, 93.51%, 93.75%, 93.88%, 94.67%, 94.00%, 94.24%, 93.94%, 93.94%
Mean Accuracy: 93.97%
Variance: 0.12%

#VGG16
Accuracies: 85.79%, 85.12%, 85.85%, 85.49%, 85.73%, 84.75%, 84.51%, 84.51%, 85.73%, 85.79%
Mean Accuracy: 85.33%
Variance: 0.28%